In [2]:
!pip3 install pandas 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd 
import networkx as nx 
import csv
import matplotlib.pyplot as plt
import numpy as np

In [47]:
df = pd.read_csv('movies.csv')
df.head()

,_id,_labels,betweenness,born,country,degree,founded,name,node_ids,node_labels,...,tagline,title,type,_start,_end,_type,date,rating,roles,summary
0,0.0,:Movie,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,...,Welcome to the Real World,The Matrix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,:Person,NaN,2000.0,NaN,7.0,NaN,Keanu Reeves,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,:Person,NaN,1967.0,NaN,3.0,NaN,Carrie-Anne Moss,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,:Person,NaN,2000.0,NaN,3.0,NaN,Laurence Fishburne,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,:Person,NaN,1960.0,NaN,5.0,NaN,Hugo Weaving,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
df['_labels']=df['_labels'].fillna('NotANode')

In [49]:
df.fillna('None', inplace=True)

In [80]:
def create_graph(df):
    G = nx.Graph()
    GRDG = nx.DiGraph()
    for i in range(len(df)):
        if(not df['_labels'][i]==':Violation') and (not df['_labels'][i]=='NotANode'):
            properties = {}
            for col in df.columns:
                if col[0]!='_' and df[col][i] != 'None':
                    properties[col] = df[col][i]
            G.add_node(int(df['_id'][i]), label=df['_labels'][i], **properties)
        if df['_labels'][i]==':Violation':
            properties = {}
            for col in df.columns:
                if col[0]!='_' and df[col][i] != 'None':
                    properties[col] = df[col][i]
            GRDG.add_node(int(df['_id'][i]), label=df['_labels'][i], **properties)
        if df['_labels'][i]=='NotANode':
            try: 
                G.add_edge(int(df['_start'][i]), int(df['_end'][i]), label=df['_type'][i])
            except:
                pass
            
            try:
                GRDG.add_edge(int(df['_start'][i]), int(df['_end'][i]), label=df['_type'][i])
            except:
                pass
    return G, GRDG


In [81]:
G, GRDG = create_graph(df)

In [82]:
nodes_to_remove = [n for n, data in G.nodes(data=True) if "label" not in data or not data["label"]]

# Remove them from the graph
G.remove_nodes_from(nodes_to_remove)

In [83]:
nodes_to_remove = [n for n, data in GRDG.nodes(data=True) if "label" not in data or not data["label"]]

# Remove them from the graph
GRDG.remove_nodes_from(nodes_to_remove)

In [84]:
for i in GRDG.nodes(data=True):
    print(i)

(171, {'label': ':Violation', 'betweenness': 0.0, 'degree': 2.0, 'node_ids': '8,0', 'node_labels': 'n,m', 'num_nodes': 2.0, 'pageRank': 0.5280744520138889, 'rel_ids': 7.0, 'rel_labls': 'r', 'type': 0.0})
(172, {'label': ':Violation', 'betweenness': 3.0, 'degree': 4.0, 'node_ids': '3,0', 'node_labels': 'n,m', 'num_nodes': 2.0, 'pageRank': 0.9609584774561832, 'rel_ids': 2.0, 'rel_labls': 'r', 'type': 0.0})
(173, {'label': ':Violation', 'betweenness': 11.0, 'degree': 7.0, 'node_ids': '1,0', 'node_labels': 'n,m', 'num_nodes': 2.0, 'pageRank': 1.453822242079721, 'rel_ids': 0.0, 'rel_labls': 'r', 'type': 0.0})
(174, {'label': ':Violation', 'betweenness': 1.0, 'degree': 3.0, 'node_ids': '3,9', 'node_labels': 'n,m', 'num_nodes': 2.0, 'pageRank': 0.7329191950514939, 'rel_ids': 10.0, 'rel_labls': 'r', 'type': 0.0})
(175, {'label': ':Violation', 'betweenness': 5.0, 'degree': 6.0, 'node_ids': '1,9', 'node_labels': 'n,m', 'num_nodes': 2.0, 'pageRank': 1.2356630584985535, 'rel_ids': 8.0, 'rel_labls'

In [85]:
pagerank = nx.pagerank(G)
degree = dict(G.degree())
betweenness = nx.betweenness_centrality(G)

# Assign values as node attributes
for node in G.nodes():
    G.nodes[node]["pagerank"] = pagerank[node]
    G.nodes[node]["degree"] = degree[node]
    G.nodes[node]["betweenness"] = betweenness[node]


In [86]:
for n in GRDG.nodes(data=True):
    nodes = n[1]['node_ids'].split(',')
    degree1 = G.nodes[int(nodes[0])]['degree']
    degree2 = G.nodes[int(nodes[1])]['degree']
    
    pr1 = G.nodes[int(nodes[0])]['pagerank']
    pr2 = G.nodes[int(nodes[1])]['pagerank']
    
    betweenness1 = G.nodes[int(nodes[0])]['betweenness']
    betweenness2 = G.nodes[int(nodes[1])]['betweenness']
    
    avg_degree = (degree1 + degree2)/2
    avg_pr = (pr1 + pr2)/2
    avg_betweenness = (betweenness1 + betweenness2)/2
    
    n[1]['avg_degree'] = avg_degree
    n[1]['avg_pr'] = avg_pr
    n[1]['avg_betweenness'] = avg_betweenness
    
    
    
    

In [87]:
nx.write_graphml(G, "graph.graphml")

nx.write_graphml(GRDG, "grdg.graphml")

In [146]:
G = nx.read_graphml("graph.graphml")
G=G.to_directed()

adjacency_matrix = nx.adjacency_matrix(G).todense()
print(adjacency_matrix)

[[0 1 1 ... 1 1 1]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


/var/folders/px/93csxpls7052_clqzsxch_8h0000gn/T/ipykernel_75779/1155152422.py:4: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adjacency_matrix = nx.adjacency_matrix(G).todense()


In [147]:
nodes = G.nodes(data=True)
node_ids_lookup = []
node_fratures = {}


In [119]:
edges = G.edges(data=True)
edge_ids_lookup = {}

In [120]:
for edge in edges:
    edge_ids_lookup[(edge[0], edge[1])] = edge[2]['label']

In [ ]:
for node in nodes: 
    try : 
        node_ids_lookup.append(node[0])
        label = node[1]['label'].strip(':')
        properties = node[1]
        properties.pop('label')
        properties.pop('degree')
        properties.pop('pagerank')
        properties.pop('betweenness')
        if(label not in node_fratures):
            node_fratures[label] ={}
            node_fratures[label]['x'] ={}
        node_fratures[label]['x'].append(list(properties.values()))
    except:
        print(node)

In [149]:
print(node_fratures)

{'Movie': [[1972.0, 'Welcome to the Real World', 'The Matrix'], [1972.0, 'Free your mind', 'The Matrix Reloaded'], [1972.0, 'Everything that has a beginning has an end', 'The Matrix Revolutions'], [1980.0, 'Evil has its winning ways', "The Devil's Advocate"], [1967.0, "In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.", 'A Few Good Men'], [1967.0, 'I feel the need, the need for speed.', 'Top Gun'], [1967.0, 'The rest of his life begins now.', 'Jerry Maguire'], [1986.0, "For some, it's the last real taste of innocence, and the first real taste of life. But for everyone, it's the time that memories are made of.", 'Stand By Me'], [1997.0, 'A comedy from the heart that goes for the throat.', 'As Good as It Gets'], [1998.0, 'After life there is more. The end is just the beginning.', 'What Dreams May Come'], [1999.0, 'First loves last. Forever.', 'Snow Falling on Cedars

In [ ]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')         

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

TypeError: list indices must be integers or slices, not str

In [172]:
for key in node_fratures.keys():
    for features in node_fratures[key]:
        for feature in features:
            if type(feature) == str:
                features.remove(feature)
                with torch.no_grad():
                    encoded_feature = model.encode([feature], convert_to_tensor=True, show_progress_bar=True)
                    encoded_feature = encoded_feature.cpu()
                print(encoded_feature)
                features = features + encoded_feature
        print(features)
        break
    break


[1972.0]


In [157]:
edge_index = [[],[]]
edge_features = []
edge_labels = []

In [158]:
for i in range(adjacency_matrix.shape[0]):
    for j in range(adjacency_matrix.shape[1]):
        if i != j:  # Exclude diagonal elements
            if adjacency_matrix[i, j] != 0:  # Check if there's an edge
                edge_index[0].append(i)
                edge_index[1].append(j)
                edge_features.append(edge_ids_lookup[(node_ids_lookup[i],node_ids_lookup[j])])
                edge_labels.append(adjacency_matrix[i, j])
                print(f"Edge {edge_ids_lookup[(node_ids_lookup[i],node_ids_lookup[j])]} from {nodes[node_ids_lookup[i]]} to {nodes[node_ids_lookup[j]]} with weight {adjacency_matrix[i, j]}")
            else:
                edge_index[0].append(i)
                edge_index[1].append(j)
                edge_labels.append(adjacency_matrix[i, j])

Edge ACTED_IN from {'released': 1972.0, 'tagline': 'Welcome to the Real World', 'title': 'The Matrix'} to {'born': 2000.0, 'name': 'Keanu Reeves'} with weight 1
Edge ACTED_IN from {'released': 1972.0, 'tagline': 'Welcome to the Real World', 'title': 'The Matrix'} to {'born': 1967.0, 'name': 'Carrie-Anne Moss'} with weight 1
Edge ACTED_IN from {'released': 1972.0, 'tagline': 'Welcome to the Real World', 'title': 'The Matrix'} to {'born': 2000.0, 'name': 'Laurence Fishburne'} with weight 1
Edge ACTED_IN from {'released': 1972.0, 'tagline': 'Welcome to the Real World', 'title': 'The Matrix'} to {'born': 1960.0, 'name': 'Hugo Weaving'} with weight 1
Edge DIRECTED from {'released': 1972.0, 'tagline': 'Welcome to the Real World', 'title': 'The Matrix'} to {'born': 1967.0, 'name': 'Lilly Wachowski'} with weight 1
Edge DIRECTED from {'released': 1972.0, 'tagline': 'Welcome to the Real World', 'title': 'The Matrix'} to {'born': 1965.0, 'name': 'Lana Wachowski'} with weight 1
Edge PRODUCED from 

In [153]:
edge_index

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  3,
  3,
  3,
  4,
  4,
  4,
  4,
  4,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  7,
  7,
  7,
  7,
  7,
  7,
  8,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  10,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  11,
  

In [154]:
edge_features

['ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'DIRECTED',
 'DIRECTED',
 'PRODUCED',
 'ACTED_IN',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'RELEASED',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'ACTED_IN',
 'DIRECTED',
 'DIRECTED',
 'DIRECTED',
 'DIRECTED',
 'WROTE',
 'WROTE',
 'PRODUCED',
 'DIRECTED',
 'DIRECTED',
 'DIRECTED',
 'DIRECTED',
 'WROTE',
 'WROTE',
 'PRODUCED',
 'PRODUCED',
 'PRODUCED'

In [160]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

numerical_edge_features = le.fit_transform(edge_features)

print(numerical_edge_features)
print(le.classes_)

[0 0 0 ... 4 4 4]
['ACTED_IN' 'DIRECTED' 'FOLLOWS' 'PRODUCED' 'RELEASED' 'REVIEWED' 'WROTE']


In [159]:
edge_labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
data = HeteroData(node_fratures, edge_index, edge_features, edge_labels)



In [173]:
df.head()

,_id,_labels,betweenness,born,country,degree,founded,name,node_ids,node_labels,...,tagline,title,type,_start,_end,_type,date,rating,roles,summary
0,0.0,:Movie,None,None,None,8.0,None,None,None,None,...,Welcome to the Real World,The Matrix,None,None,None,None,None,None,None,None
1,1.0,:Person,None,2000.0,None,7.0,None,Keanu Reeves,None,None,...,None,None,None,None,None,None,None,None,None,None
2,2.0,:Person,None,1967.0,None,3.0,None,Carrie-Anne Moss,None,None,...,None,None,None,None,None,None,None,None,None,None
3,3.0,:Person,None,2000.0,None,3.0,None,Laurence Fishburne,None,None,...,None,None,None,None,None,None,None,None,None,None
4,4.0,:Person,None,1960.0,None,5.0,None,Hugo Weaving,None,None,...,None,None,None,None,None,None,None,None,None,None


In [183]:
df.columns

Index(['_id', '_labels', 'betweenness', 'born', 'country', 'degree', 'founded',
       'name', 'node_ids', 'node_labels', 'num_nodes', 'pageRank', 'rel_ids',
       'rel_labls', 'released', 'tagline', 'title', 'type', '_start', '_end',
       '_type', 'date', 'rating', 'roles', 'summary'],
      dtype='object')

In [250]:
df = pd.read_csv('./movies.csv')
df['_labels']=df['_labels'].fillna('NotANode')

In [ ]:
violation_df = df[df['_labels'] == 'Violation']
violation_df=violation_df.dropna(axis=1)
violation_df.head()


,_id,_labels,betweenness,degree,node_ids,node_labels,num_nodes,pageRank,rel_ids,rel_labls,type
171,171.0,Violation,0.0,2.0,"8,0","n,m",2.0,0.528074,7.0,r,0.0
172,172.0,Violation,3.0,4.0,"3,0","n,m",2.0,0.960958,2.0,r,0.0
173,173.0,Violation,11.0,7.0,"1,0","n,m",2.0,1.453822,0.0,r,0.0
174,174.0,Violation,1.0,3.0,"3,9","n,m",2.0,0.732919,10.0,r,0.0
175,175.0,Violation,5.0,6.0,"1,9","n,m",2.0,1.235663,8.0,r,0.0


In [340]:
# Extract and flatten node_ids
node_list = violation_df['node_ids'].str.split(',').sum()
violation_nodes = sorted(set(map(int, node_list)))

# Extract and flatten rel_ids
rel_list = violation_df['rel_ids'].astype(str).str.split(',').sum()
violation_rels = sorted(set(map(float, rel_list)))

print("violation Nodes:", violation_nodes)
print("violation Relationships:", violation_rels)

violation Nodes: [0, 1, 3, 8, 9, 10, 11, 13, 15, 17, 19, 20, 21, 22, 23, 25, 29, 31, 33, 37, 38, 40, 41, 43, 44, 46, 52, 56, 62, 67, 73, 78, 81, 85, 87, 95, 97, 100, 105, 106, 111, 116, 117, 120, 141, 152, 205, 208, 209, 211, 215, 221, 230, 231, 233, 234, 235, 236]
violation Relationships: [0.0, 2.0, 7.0, 8.0, 10.0, 15.0, 17.0, 22.0, 24.0, 27.0, 29.0, 30.0, 31.0, 32.0, 33.0, 35.0, 42.0, 44.0, 49.0, 50.0, 52.0, 53.0, 55.0, 56.0, 114.0, 132.0, 139.0, 152.0, 155.0, 197.0, 216.0, 388.0, 394.0, 415.0, 454.0, 457.0, 469.0, 482.0, 504.0, 506.0, 522.0, 528.0, 538.0, 546.0, 552.0, 554.0, 555.0, 556.0, 557.0, 558.0, 560.0, 562.0, 564.0, 572.0, 574.0, 583.0, 584.0, 586.0, 588.0, 589.0, 590.0, 602.0, 614.0, 616.0, 622.0, 624.0, 626.0, 630.0, 638.0, 641.0, 642.0, 644.0, 646.0, 648.0, 666.0, 670.0, 688.0, 692.0, 694.0, 704.0, 720.0, 744.0, 750.0, 758.0, 788.0, 792.0, 796.0, 802.0, 804.0, 805.0, 806.0, 807.0, 808.0, 818.0, 822.0, 826.0, 840.0, 846.0, 852.0, 854.0, 856.0, 858.0, 862.0, 880.0, 888.0, 8

In [251]:
df['_labels'] = df['_labels'].apply(lambda x: x.strip(':'))

In [252]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np


# Fit on global labels
label_encoder = OneHotEncoder(sparse_output=False)
label_encoder.fit(df[['_labels']].apply(lambda labels: [label for label in labels if label not in ['Violation', 'NotANode']]))


OneHotEncoder(sparse_output=False)

In [341]:
#Person

label_df = df[df['_labels'] == 'Person']
person_df = label_df.drop(columns=['degree', 'pageRank', 'betweenness'])
person_df=person_df.dropna(axis=1)
person_df['_id'] = person_df['_id'].astype(int)
print(person_df.head())


   _id _labels    born                name
1    1  Person  2000.0        Keanu Reeves
2    2  Person  1967.0    Carrie-Anne Moss
3    3  Person  2000.0  Laurence Fishburne
4    4  Person  1960.0        Hugo Weaving
5    5  Person  1967.0     Lilly Wachowski


In [ ]:
# Ensure the '_id' column is the same type (int)


# Filter based on whether _id is in unique_nodes
test_person_df = person_df[person_df['_id'].isin(violation_nodes)]
print(test_person_df.shape)
train_person_df = person_df[~person_df['_id'].isin(violation_nodes)]
print(train_person_df.shape)
matching_indexes = df_people[df_people['_id'].isin(violation_nodes)].index.tolist()

(21, 4)
(112, 4)


In [254]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sentence_transformers import SentenceTransformer


In [ ]:
label_onehot = label_encoder.transform(train_person_df[['_labels']])
scaler = MinMaxScaler()
born_scaled = scaler.fit_transform(train_person_df[['born']])
model = SentenceTransformer('all-MiniLM-L6-v2')  # Or any other
name_embeddings = model.encode(train_person_df['name'].tolist(), convert_to_numpy=True)
person_features = np.hstack([label_onehot, born_scaled, name_embeddings])

In [345]:
person_features

array([[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         4.26391102e-02,  6.32569790e-02, -6.00491476e-04],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
        -6.36293739e-02,  1.16768964e-02, -4.25202288e-02],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
        -3.55457887e-02,  4.66854237e-02, -1.19062830e-02],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
        -5.05581237e-02, -3.45533118e-02, -1.10844709e-02],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
        -4.43958379e-02,  4.29285914e-02, -8.06264132e-02],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
        -4.92963083e-02,  4.01831530e-02, -4.91667390e-02]])

In [ ]:
label_onehot = label_encoder.transform(person_df[['_labels']])
scaler = MinMaxScaler()
born_scaled = scaler.fit_transform(person_df[['born']])
model = SentenceTransformer('all-MiniLM-L6-v2')  # Or any other
name_embeddings = model.encode(person_df['name'].tolist(), convert_to_numpy=True)
person_features = np.hstack([label_onehot, born_scaled, name_embeddings])

In [ ]:
parson_mapping={}
for count,id in enumerate(person_df['_id']):
    parson_mapping[id]=count    

In [263]:
person_x = torch.tensor(person_features, dtype=torch.float)


In [264]:
from torch_geometric.data import HeteroData
data = HeteroData()


In [265]:
#Movie

label_df = df[df['_labels'] == 'Movie']
movie_df = label_df.drop(columns=['degree', 'pageRank', 'betweenness'])
movie_df=movie_df.dropna(axis=1)
print(movie_df.head())


     _id _labels  released                   title
0    0.0   Movie    1972.0              The Matrix
9    9.0   Movie    1972.0     The Matrix Reloaded
10  10.0   Movie    1972.0  The Matrix Revolutions
11  11.0   Movie    1980.0    The Devil's Advocate
15  15.0   Movie    1967.0          A Few Good Men


In [325]:
movie_df.shape

(38, 4)

In [266]:
label_onehot = label_encoder.transform(movie_df[['_labels']])
print(label_onehot)
released_scaled = scaler.fit_transform(movie_df[['released']])

title_embeddings = model.encode(movie_df['title'].tolist(), convert_to_numpy=True)

movie_features = np.hstack([label_onehot, released_scaled, title_embeddings])

movie_x = torch.tensor(movie_features, dtype=torch.float)



[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [326]:
movie_x.shape

torch.Size([38, 388])

In [267]:
movie_mapping={}
for count,id in enumerate(movie_df['_id']):
    movie_mapping[id]=count    

In [268]:
#Cinema

label_df = df[df['_labels'] == 'Cinema']
cinema_df = label_df.drop(columns=['degree', 'pageRank', 'betweenness'])
cinema_df=cinema_df.dropna(axis=1)
print(cinema_df.head())


       _id _labels        country  founded                    name
202  202.0  Cinema         Canada   1979.0  Cineplex Odeon Cinemas
203  203.0  Cinema         Canada   1920.0          Famous Players
204  204.0  Cinema  United States   1920.0            AMC Theatres
205  205.0  Cinema  United States   1989.0           Regal Cinemas
206  206.0  Cinema      Australia   1909.0           Hoyts Cinemas


In [ ]:
label_onehot = label_encoder.transform(cinema_df[['_labels']])
print(label_onehot)
country_encoder = OneHotEncoder(sparse_output=False)
country_onehot = country_encoder.fit_transform(cinema_df[['country']])

founded_scaled = scaler.fit_transform(cinema_df[['founded']])

cinema_name_embeddings = model.encode(cinema_df['name'].tolist(), convert_to_numpy=True)

cinema_features = np.hstack([label_onehot, country_onehot, founded_scaled, cinema_name_embeddings])

cinema_x = torch.tensor(cinema_features, dtype=torch.float)


[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [270]:
cinema_mapping={}
for count,id in enumerate(cinema_df['_id']):
    cinema_mapping[id]=count    

In [327]:
data['person'].x = person_x
data['movie'].x = movie_x
data['cinema'].x = cinema_x


In [328]:
def get_node_type(node_id):
    if node_id in parson_mapping:
        return 'person'
    elif node_id in movie_mapping:
        return 'movie'
    elif node_id in cinema_mapping:
        return 'cinema'
    else:
        raise ValueError(f'Unknown node ID: {node_id}')


In [329]:
from collections import defaultdict

In [330]:
edges_df = df[df['_labels'] == 'NotANode'].dropna(axis=1)
edges_df = edges_df[edges_df['_type'] != 'BELONGS']
edges_df = edges_df[edges_df['_type'] != 'INTERSECT']

edges_df

,_labels,_start,_end,_type
329,NotANode,1.0,0.0,ACTED_IN
330,NotANode,2.0,0.0,ACTED_IN
331,NotANode,3.0,0.0,ACTED_IN
332,NotANode,4.0,0.0,ACTED_IN
333,NotANode,5.0,0.0,DIRECTED
...,...,...,...,...
1457,NotANode,116.0,227.0,RELEASED
1458,NotANode,116.0,234.0,RELEASED
1459,NotANode,116.0,235.0,RELEASED
1460,NotANode,116.0,232.0,RELEASED


In [331]:


# To collect edges by (src_type, relation_type, dst_type)
edge_dict = defaultdict(list)

for _, row in edges_df.iterrows():
    src_id = row['_start']
    dst_id = row['_end']
    rel_type = row['_type']

    #try:
    src_type = get_node_type(src_id)
    dst_type = get_node_type(dst_id)


    src_idx = eval(f"{src_type}_id_map")[src_id]
    dst_idx = eval(f"{dst_type}_id_map")[dst_id]

    

    edge_dict[(src_type, rel_type, dst_type)].append((src_idx, dst_idx))
    #except:
    #    pass
    
edge_dict

defaultdict(list,
            {('person', 'ACTED_IN', 'movie'): [(0, 0),
              (1, 0),
              (2, 0),
              (3, 0),
              (7, 0),
              (0, 1),
              (1, 1),
              (2, 1),
              (3, 1),
              (0, 2),
              (1, 2),
              (2, 2),
              (3, 2),
              (0, 3),
              (8, 3),
              (9, 3),
              (11, 4),
              (12, 4),
              (13, 4),
              (14, 4),
              (15, 4),
              (16, 4),
              (17, 4),
              (18, 4),
              (19, 4),
              (20, 4),
              (21, 4),
              (23, 4),
              (11, 5),
              (24, 5),
              (25, 5),
              (26, 5),
              (27, 5),
              (28, 5),
              (11, 6),
              (17, 6),
              (31, 6),
              (32, 6),
              (33, 6),
              (34, 6),
              (35, 6),
              (36, 6),

In [303]:
for (src_type, rel_type, dst_type), edge_list in edge_dict.items():
    src_list, dst_list = zip(*edge_list)
    edge_index = torch.tensor([src_list, dst_list], dtype=torch.long)
    data[(src_type, rel_type, dst_type)].edge_index = edge_index
    
    


In [307]:
data.edge_index_dict

{('person',
  'ACTED_IN',
  'movie'): tensor([[  0,   1,   2,   3,   7,   0,   1,   2,   3,   0,   1,   2,   3,   0,
            8,   9,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  23,
           11,  24,  25,  26,  27,  28,  11,  17,  31,  32,  33,  34,  35,  36,
           37,  41,  39,  33,  40,  42,  15,  43,  12,  44,  45,  17,  50,  17,
           47,  48,  49,  52,  53,  48,  54,  59,  28,  45,  56,  57,  58,  59,
           28,  61,  62,  63,  64,  59,  28,  66,  67,  28,  68,  69,  59,  70,
            8,   0,  71,  72,  73,  43,  75,  76,  58,  50,  66,  72,  78,  79,
           72,   0,  80,  81,  82,  59,   3,  84,  85,  59,  89,  90,  91,   3,
           93,  94,  95,  96,  97,  98,  99, 100, 101, 102,  96, 102, 103,  53,
           96,  59, 104, 105,  35,  54, 106, 107, 108, 110, 111,  14, 112, 106,
           12, 113,  19, 114,  59,  14, 115, 116, 107, 116,  44,  76, 117,  59,
           44,  12, 113,  12, 121,   0,  50, 112,  59, 124, 117,  59,  59, 126,
   

In [297]:
data['cinema'].x.shape
data['person'].x.shape
data['movie'].x.shape

torch.Size([36, 406])

In [336]:
torch.save(data, 'heterodata_onehot_no_binning.pt')